In [ ]:
import pandas as pd
import glob
import numpy as np

from keras.models import Sequential
from keras.layers import Activation
from keras.layers.core import Dense, Flatten
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
import tensorflow as tf

import time


import matplotlib.pyplot as plt

from sklearn.model_selection import KFold# FUNCIONES GRAFICAS:

# FUNCIONES  DE LAS GRAFICAS:

In [ ]:
def plot_history1(history):
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch

    plt.figure()
    plt.xlabel('Epoch')
    plt.ylabel('Mean Abs Error [MPG]')
    plt.plot(hist['epoch'], hist['mae'],
           label='Train Error')
    plt.plot(hist['epoch'], hist['val_mae'],
           label = 'Val Error')
    plt.ylim([0,plt.ylim()[1]])
    plt.legend()
    plt.show()

In [ ]:
def plot_history2(history):
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch

    plt.figure()
    plt.xlabel('Epoch')
    plt.ylabel('Mean Square Error [$MPG^2$]')
    plt.plot(hist['epoch'], hist['mse'],
           label='Train Error')
    plt.plot(hist['epoch'], hist['val_mse'],
           label = 'Val Error')
    plt.ylim([0,plt.ylim()[1]])
    plt.legend()
    plt.show()

In [ ]:
def plot_comparacion(test_x,test_y):
    plt.scatter( test_y , test_x )
    plt.xlabel('True Values [MPG]')
    plt.ylabel('Predictions [MPG]')
    plt.axis('equal')
    plt.axis('square')
    plt.xlim([0,plt.xlim()[1]])
    plt.ylim([0,plt.ylim()[1]])
    _ = plt.plot(test_y , test_y , color = 'orange')
    plt.show()

In [ ]:
def plot_histograma(test_x,test_y):
    error = test_x - test_y
    plt.hist(error, bins = 25)
    plt.xlabel("Prediction Error [MPG]")
    _ = plt.ylabel("Count")
    plt.show()

# Preparacion de los datos NN simple

In [ ]:
a_x2 = np.loadtxt('datos_x_2.txt')
a_y2 = np.loadtxt('datos_y_2.txt')

In [ ]:
x_m = x_data.reshape(( int(len(x_data)/(72)), 6 , 12 , 1 ))
y_m = y_data.reshape(( int(len(y_data)/(72)), 6 , 12 , 1 ))

In [ ]:
x = x_m
y = y_m

# Definimos el metodo k-folds

In [ ]:
# Define the K-fold Cross Validator
num_folds = 4
kfold = KFold(n_splits=num_folds, shuffle=True)
#Definir valores de numero de capas y numero de fold en el que estamos trabajando:
fold_no = 1
tipo = 'CONV2D_2'

In [ ]:

# K-fold Cross Validation model evaluation
acc_per_fold = []
loss_per_fold = []

for train, test in kfold.split(x, y):
    
    modelo = Sequential()
    
    modelo.add(Conv2D(72 , kernel_size=(1,1), activation='relu', input_shape = ((6,12,1))))

    modelo.add(MaxPooling2D(pool_size=(2,2))) # layer to pool and reduce the dimensionality of the data.
    modelo.add(Conv2D(30 , kernel_size=(1,1), activation='relu'))

    #Comentar estas dos capas -----------------------------------------------------------------------------
    modelo.add(MaxPooling2D(pool_size=(2, 2))) # layer to pool and reduce the dimensionality of the data.
    modelo.add(Conv2D(15 , kernel_size=(1,1), activation='relu'))

    modelo.add(UpSampling2D(size=( 3, 2))) # layer to pool and reduce the dimensionality of the data.
    modelo.add(Conv2D(30 , kernel_size=(1,1), activation='relu'))
    #------------------------------------------------------------------------------------------------------
    
    modelo.add(UpSampling2D(size=( 2, 2))) # layer to pool and reduce the dimensionality of the data.
    modelo.add(Conv2D(1 , kernel_size=(1,1), activation='relu'))


    modelo.add(Flatten()) #Flatten the output from the convolutional layer and pass it to a Dense layer
    modelo.add(Dense(72, activation='relu'))
    modelo.add(Dense(30, activation='relu'))
    modelo.add(Dense(15, activation='relu'))
    modelo.add(Dense(30, activation='relu'))
    modelo.add(Dense(72))
    modelo.summary()

    # COMPILACION DEL MODELO
    optimizer = tf.keras.optimizers.RMSprop(0.001)

    modelo.compile(loss='mse',
                optimizer=optimizer,
                metrics=['mae', 'mse'])


    
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')

    # ENTRENAMIENTO DE LOS DATOS:
    
    x_train_1 = x[train][:int(len(x[train])/4 * 3)]
    x_val = x[train][int(len(x[train])/4 * 3):]
    y_train_1 = y[train][:int(len(y[train])/4 * 3)]
    y_val = y[train][int(len(y[train])/4 * 3):]
    

    
    history = modelo.fit(x_train_1, y_train_1,
                         validation_data=(x_val, y_val),
                         batch_size=1,
                         epochs=100,
                         verbose=0)
    
    #GRAFICAS DEL PRIMER ENTRENAMIENTO:
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch
    hist.to_csv('Hist_' + str(fold_no) + '_' + str(tipo) + '.csv')
    
    plot_history1(history)
    time.sleep(10) 
    
    plot_history2(history)
    time.sleep(10) 
    

    # ENTRENAMIENTO DEL MODELO DE SEGUNDA VEZ 
    print('    ')
    print('Segundo entrenamiento del modelo')
    print('    ')
    a = modelo.fit(x[train], y[train],
                   batch_size=1,
                   epochs=100,
                   verbose=0)

    hist2 = pd.DataFrame(a.history)
    hist2['epoch'] = a.epoch
    
    
    # EVALUAMOS EL MODELO
    scores = modelo.evaluate(x[test], y[test], verbose=0)
    print('    ')
    print('    ')
    print('    ')
    print(f'Score for fold {fold_no}: {modelo.metrics_names[0]} of {scores[0]}; {modelo.metrics_names[1]} of {scores[1]}')
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])
    
    #REALIZAMOS LAS PREDICCIONES:
    test_x = modelo.predict(x[test])
    test_y = y[test]
    
    #MOSTRAMOS UNA DE LA SPREDICCIONES PARA COMPARAR REAL-PREDICCION:
    print('Prediccion: ')
    print(test_x[1])
    print('Real: ')
    print(test_y[1])
    
    #GRAFICAS DEL SEGUNDO ENTRENAMIENTO:
    plot_comparacion(test_x, test_y)
    
    plot_histograma(test_x,test_y)
    
    print('    ')
    print('    ')
    print('    ')
    print('Terminamos hoja numero --------------------------------------------------> ' , fold_no )
    print('    ')
    print('    ')
    print('    ')
    
    # INCREMENTAMOS EL NUMERO DE HOJAS:
    
    fold_no = fold_no + 1

